In [1]:
import pandas as pd
df1 = pd.read_csv('./data/cierres_diarios_M1.csv', parse_dates=['Date'], index_col='Date')
df2 = pd.read_csv('./data/cierres_diarios_M2.csv', parse_dates=['Date'], index_col='Date')
df3 = pd.read_csv('./data/cierres_diarios_M3.csv', parse_dates=['Date'], index_col='Date')

In [2]:
# ============================================================
#  ESCENARIO M (window=60, horizon=5) — MODELO TRANSFORMER
#  - Usa df1, df2, df3 ya cargados (longitud: ≥ 65 filas)
#  - Escalado consistente con el entrenamiento (MinMax a [0,1])
#  - Evalúa SOLO la primera ventana (60→5) de cada df (M1, M2, M3)
# ============================================================

# Eliminación de nulos
df1.ffill(inplace=True); df1.bfill(inplace=True)
df2.ffill(inplace=True); df2.bfill(inplace=True)
df3.ffill(inplace=True); df3.bfill(inplace=True)

import pandas as pd
import numpy as np
import tensorflow as tf
from joblib import load
from typing import List

# Cargar scaler y columnas base (mismo orden que en entrenamiento)
scaler = load("scaler_modelos.joblib")

# Orden de columnas usado en entrenamiento
feature_order = list(scaler.feature_names_in_)

# Filtramos/ordenamos exactamente como en entrenamiento
df1 = df1[feature_order].copy()
df2 = df2[feature_order].copy()
df3 = df3[feature_order].copy()

# Escalado
df1_scaled = pd.DataFrame(scaler.transform(df1[feature_order]), index=df1.index, columns=feature_order).astype("float32")
df2_scaled = pd.DataFrame(scaler.transform(df2[feature_order]), index=df2.index, columns=feature_order).astype("float32")
df3_scaled = pd.DataFrame(scaler.transform(df3[feature_order]), index=df3.index, columns=feature_order).astype("float32")

# --- Carga del modelo Transformer (60→5) ---
from tensorflow.keras.models import load_model

path_m = "./models_transformer_huber_sweep/transformer_huber_w60_h5_delta0_1_M.keras"
delta_m = 0.1

# (Opcional) detectar capas personalizadas registradas
def maybe_get_custom_objects():
    names = [
        "PositionalEmbedding", "Custom>PositionalEmbedding",
        "TransformerBlock", "Custom>TransformerBlock",
        # añade otros nombres si registraste más capas/bloques
    ]
    custom = {}
    for n in names:
        obj = globals().get(n, None)
        if obj is not None:
            custom[n] = obj
    return custom if custom else None

custom_objects = maybe_get_custom_objects()

try:
    if custom_objects:
        model_m = load_model(path_m, compile=False, custom_objects=custom_objects)
    else:
        model_m = load_model(path_m, compile=False)
except Exception as e:
    raise RuntimeError(
        "No se pudo cargar el Transformer. Si usa capas personalizadas, pásalas en custom_objects.\n"
        f"Error original: {e}"
    )

# === Helpers de dataset (siempre datos ESCALADOS) ===
def make_dataset(data_scaled, window_size, horizon, batch_size=32, shuffle=False):
    ds = tf.keras.preprocessing.timeseries_dataset_from_array(
        data=data_scaled.values,
        targets=None,
        sequence_length=window_size + horizon,
        sequence_stride=1,
        batch_size=batch_size,
        shuffle=shuffle
    )
    return ds.map(
        lambda seq: (
            tf.cast(seq[:, :window_size, :], tf.float32),   # X: (B,w,F)
            tf.cast(seq[:, window_size:, :], tf.float32)    # y: (B,h,F)
        )
    )

window_size = 60
horizon = 5

# Comprobación de longitud mínima (≥ w + h)
def ensure_min_rows(df_scaled, name, w, h):
    if len(df_scaled) < w + h:
        raise ValueError(f"{name}: se necesitan al menos {w+h} filas para {w}→{h}; hay {len(df_scaled)}.")

ensure_min_rows(df1_scaled, "df1", window_size, horizon)
ensure_min_rows(df2_scaled, "df2", window_size, horizon)
ensure_min_rows(df3_scaled, "df3", window_size, horizon)

# Evaluar SOLO la primera (60→5): 65 filas = 60 + 5
ds1 = make_dataset(df1_scaled.iloc[:window_size+horizon], window_size, horizon, batch_size=1, shuffle=False)
ds2 = make_dataset(df2_scaled.iloc[:window_size+horizon], window_size, horizon, batch_size=1, shuffle=False)
ds3 = make_dataset(df3_scaled.iloc[:window_size+horizon], window_size, horizon, batch_size=1, shuffle=False)

# === Métricas y pérdidas ===
def _eps_tag(x: float) -> str:
    return str(x).replace('.', '_')

def log_cosh_metric(y_true, y_pred):
    e = tf.cast(y_pred, tf.float32) - tf.cast(y_true, tf.float32)
    ae = tf.abs(e)
    return tf.reduce_mean(ae + tf.nn.softplus(-2.0 * ae) - tf.math.log(2.0))
log_cosh_metric.__name__ = "log_cosh"

def make_huber_loss(delta: float):
    base = tf.keras.losses.Huber(delta=float(delta))
    def huber_loss(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32); y_pred = tf.cast(y_pred, tf.float32)
        return base(y_true, y_pred)
    huber_loss.__name__ = f"huber_delta_{_eps_tag(delta)}"
    return huber_loss

def make_within_eps_vector_metric(eps_vec: np.ndarray, tag: str):
    """eps_vec shape (F,) en la MISMA escala que y_true/y_pred (normalizada)."""
    eps_tf = tf.constant(eps_vec.astype(np.float32), dtype=tf.float32)  # (F,)
    def within_eps(y_true, y_pred):
        diff = tf.abs(tf.cast(y_pred, tf.float32) - tf.cast(y_true, tf.float32))  # (B,H,F)
        thr  = eps_tf[tf.newaxis, tf.newaxis, :]                                   # (1,1,F)
        hit  = tf.cast(diff <= thr, tf.float32)
        return tf.reduce_mean(hit)
    within_eps.__name__ = tag
    return within_eps

def build_within_metrics_minmax(scaler, eps_list: List[float], n_features: int):
    """
    MinMaxScaler: eps = porcentaje * data_range_ por feature.
    En escala [0,1], el 'porcentaje del rango' coincide con el valor eps.
    """
    metrics = [log_cosh_metric]
    if not hasattr(scaler, "data_range_"):
        raise ValueError("Se esperaba un MinMaxScaler con atributo data_range_.")
    if len(scaler.data_range_) != n_features:
        raise ValueError("scaler.data_range_ no coincide con n_features.")
    for e in eps_list:
        eps_vec = np.full((n_features,), float(e), dtype=np.float32)
        tag = f"within_eps_{_eps_tag(e)}"
        metrics.append(make_within_eps_vector_metric(eps_vec, tag))
    return metrics

def compute_autc_from_results(res: dict, eps_list: List[float]) -> float:
    """
    Calcula AUTC normalizando por el rango [ε_min, ε_max], ∈ [0,1].
    Toma los valores within_eps_* devueltos por model.evaluate(return_dict=True).
    """
    eps = np.array(sorted(eps_list), dtype=np.float32)
    acc = np.array([res.get(f"within_eps_{str(e).replace('.','_')}", np.nan) for e in eps],
                   dtype=np.float32)
    mask = np.isfinite(acc)
    if mask.sum() < 2:
        return float("nan")
    eps = eps[mask]; acc = acc[mask]
    return float(np.trapz(acc, eps) / (eps[-1] - eps[0]))

# === Compilar el modelo con las métricas within-ε correctas (escala normalizada) ===
EPS_LIST = [0.005, 0.01, 0.02, 0.05, 0.1]
n_features = len(feature_order)
metrics = build_within_metrics_minmax(scaler, EPS_LIST, n_features=n_features)
model_m.compile(optimizer="adam", loss=make_huber_loss(delta_m), metrics=metrics)

def evaluate_and_print(tag, dataset):
    res = model_m.evaluate(dataset, return_dict=True, verbose=0)
    print(f"\n=== Resultados {tag} (60→5) ===")
    print(f"  Huber (loss):              {res.get('loss', float('nan')):.6f}")
    print(f"  log_cosh:                  {res.get('log_cosh', float('nan')):.6f}")
    for e in EPS_LIST:
        key = f"within_eps_{str(e).replace('.','_')}"
        print(f"  {key:26s}: {res.get(key, float('nan')):.6f}")
    autc = compute_autc_from_results(res, EPS_LIST)
    print(f"  AUTC[{min(EPS_LIST):.3f}–{max(EPS_LIST):.3f}]:         {autc:.6f}")
    return res

# === Evaluación de M1, M2 y M3 (primera ventana 60→5 de cada df) ===
res1 = evaluate_and_print("M1", ds1)
res2 = evaluate_and_print("M2", ds2)
res3 = evaluate_and_print("M3", ds3)

# === Ventana ÚNICA (60→5) por cada M para inspección puntual ===
def single_window_arrays(df_scaled, w=60, h=5):
    if len(df_scaled) < w + h:
        raise ValueError(f"No hay suficientes días hábiles para {w}→{h}.")
    X = df_scaled.iloc[0:w].to_numpy()[np.newaxis, :, :]   # (1,w,F)
    Y = df_scaled.iloc[w:w+h].to_numpy()[np.newaxis, :, :] # (1,h,F)
    return X.astype("float32"), Y.astype("float32")

# --- 1) Tabla con todas las métricas por escenario ---
def res_to_series(tag, res, eps_list):
    d = {
        "loss": res.get("loss", np.nan),
        "log_cosh": res.get("log_cosh", np.nan),
        "AUTC": compute_autc_from_results(res, eps_list),
    }
    for e in eps_list:
        key = f"within_eps_{str(e).replace('.','_')}"
        d[key] = res.get(key, np.nan)
    s = pd.Series(d, name=tag)
    return s

m1 = res_to_series("M1", res1, EPS_LIST)
m2 = res_to_series("M2", res2, EPS_LIST)
m3 = res_to_series("M3", res3, EPS_LIST)

df_all = pd.concat([m1, m2, m3], axis=1).T  # filas: escenarios; cols: métricas
df_mean = df_all.mean(axis=0)

print("\n==== Tabla de métricas por escenario ====")
print(df_all.round(6))
print("\n==== Medias (M1,M2,M3) ====")
print(df_mean.round(6))


2025-09-22 20:14:23.026060: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2025-09-22 20:14:23.026089: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-09-22 20:14:23.026096: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-09-22 20:14:23.026115: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-09-22 20:14:23.026123: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2025-09-22 20:14:23.603880: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.



=== Resultados M1 (60→5) ===
  Huber (loss):              0.337530
  log_cosh:                  2.987760
  within_eps_0_005          : 0.000000
  within_eps_0_01           : 0.008000
  within_eps_0_02           : 0.016000
  within_eps_0_05           : 0.056000
  within_eps_0_1            : 0.096000
  AUTC[0.005–0.100]:         0.052842

=== Resultados M2 (60→5) ===
  Huber (loss):              0.310560
  log_cosh:                  2.700321
  within_eps_0_005          : 0.000000
  within_eps_0_01           : 0.008000
  within_eps_0_02           : 0.016000
  within_eps_0_05           : 0.040000
  within_eps_0_1            : 0.072000
  AUTC[0.005–0.100]:         0.039789

=== Resultados M3 (60→5) ===
  Huber (loss):              0.295122
  log_cosh:                  2.520308
  within_eps_0_005          : 0.000000
  within_eps_0_01           : 0.008000
  within_eps_0_02           : 0.008000
  within_eps_0_05           : 0.008000
  within_eps_0_1            : 0.032000
  AUTC[0.005–0.100]: 